In [ ]:
!pip install --quiet openai pandas numpy scikit-learn matplotlib joblib

import os
import time
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import importlib
import warnings

try:
    from openai import OpenAI
    client = OpenAI()
    USE_NEW_SDK = True
    print("✅ Using new OpenAI SDK (client = OpenAI())")

    def get_embedding(text, model="text-embedding-ada-002"):
        response = client.embeddings.create(model=model, input=text)
        return response.data[0].embedding

except ImportError:
    import openai
    USE_NEW_SDK = False
    print("⚠️ Using legacy OpenAI SDK (openai.Embedding.create)")

    def get_embedding(text, model="text-embedding-ada-002"):
        response = openai.Embedding.create(model=model, input=text)
        return response['data'][0]['embedding']

os.environ["OPENAI_API_KEY"] = "Your_api_key"

if USE_NEW_SDK:
    client.api_key = os.environ["OPENAI_API_KEY"]
else:
    openai.api_key = os.environ["OPENAI_API_KEY"]


data = [
    {"name": "Boho Dress", "desc": "Flowy, earthy tones for festival vibes and carefree days.", "vibes": ["boho", "carefree"]},
    {"name": "Urban Jacket", "desc": "Sleek leather with a confident, city-night aesthetic.", "vibes": ["urban", "chic"]},
    {"name": "Cozy Sweater", "desc": "Soft knit with warm tones for relaxing weekends.", "vibes": ["cozy", "casual"]},
    {"name": "Sporty Joggers", "desc": "Lightweight and flexible pants for active mornings.", "vibes": ["sporty", "minimal"]},
    {"name": "Elegant Blazer", "desc": "Structured silhouette for modern office wear.", "vibes": ["formal", "minimal"]},
    {"name": "Beach Shorts", "desc": "Bright colors and relaxed fit for summer escapes.", "vibes": ["casual", "tropical"]},
    {"name": "Grunge Tee", "desc": "Distressed print and raw edge finish for edgy streetwear.", "vibes": ["grunge", "urban"]},
    {"name": "Classic Denim", "desc": "Timeless jeans that fit every mood and style.", "vibes": ["classic", "casual"]},
    {"name": "Minimal Sandals", "desc": "Clean lines and neutral tones for laid-back days.", "vibes": ["minimal", "boho"]},
]

df = pd.DataFrame(data)
print("🧺 Product catalog ready:", len(df), "items")
df.head()

CACHE_PATH = "embeddings_cache.pkl"

if os.path.exists(CACHE_PATH):
    df_cached, cache_info = joblib.load(CACHE_PATH)
    if "embedding" in df_cached.columns:
        df = df_cached
        print(f"✅ Loaded cached embeddings for {len(df)} products (cached on {cache_info.get('generated_on','unknown')})")
    else:
        print("⚠️ Cache found but no embeddings, will regenerate.")
        cache_info = {"generated_on": time.ctime()}
else:
    print("⚠️ No cache found. Will generate new embeddings...")
    cache_info = {"generated_on": time.ctime()}


if "embedding" not in df.columns or df["embedding"].isnull().any():
    print("⏳ Generating embeddings (this may take a minute)...")
    df["embedding"] = df["desc"].apply(get_embedding)
    joblib.dump((df, cache_info), CACHE_PATH)
    print("💾 Embeddings cached successfully at:", CACHE_PATH)
else:
    print("✅ Embeddings already exist, skipping generation.")


def vibe_matcher(query, top_k=3):
    start = time.time()
    query_emb = get_embedding(query)
    sims = cosine_similarity([query_emb], df["embedding"].to_list())[0]
    df["similarity"] = sims
    results = df.sort_values("similarity", ascending=False).head(top_k)
    latency = time.time() - start
    return results[["name", "desc", "similarity"]], latency


queries = [
    "energetic urban chic",
    "relaxed vacation mood",
    "office professional"
]

metrics = []
for q in queries:
    results, latency = vibe_matcher(q)
    print(f"\n🔍 Query: {q}")
    print(results)
    metrics.append({
        "query": q,
        "top1_score": results.iloc[0]["similarity"],
        "latency": latency,
        "good_match": results.iloc[0]["similarity"] > 0.7
    })

metrics_df = pd.DataFrame(metrics)
print("\n📊 Query Metrics:")
print(metrics_df)


plt.figure(figsize=(6,4))
plt.bar(metrics_df["query"], metrics_df["latency"], color="skyblue")
plt.ylabel("Seconds")
plt.title("Vibe Matcher Latency per Query")
plt.xticks(rotation=20)
plt.show()


print("""
📌 Reflection:
- ✅ Cached embeddings reduce rerun latency dramatically.
- ✅ Handles both old and new OpenAI SDKs.
""")
